In [1]:
import sys
import os
sys.path.append(os.path.join("..","COVID-Argentina"))
import model 
import geopandas as gdp
import pandas as pd
import numpy as np
import scipy
from model.sim import location, state, contacts

In [2]:
crs = {'epsg:3857'}

In [3]:
file1 = gdp.read_file("./shp/Barrios_info_DANE.shp") 
file1['SEXO_TOT_1'].sum()

551559

In [4]:
people = gdp.read_file("./shp/Censo_personas_manzanas_2018.shp") 

In [18]:
people

,DEPTO,MPIO,U_DPTO,U_MPIO,UA_CLASE,UA2_CPOB,UASECT_URB,UASECC_URB,UA_MANZANA,ID_UNIFICA,...,Shape__L_1,h,EDAD_20_60,pob,BARRIO,COMUNA,GlobalID,SHAPE_Leng,SHAPE_Area,geometry
0,ATLÁNTICO,SOLEDAD,08,758,1,000,0001,01,05,0875810000000000010105,...,2443.258257,1,0,0,Zona Industrial Aeropuerto,Oriental,{5B4B3BD7-50A7-4484-B0E5-3029B74A7B8E},2443.258062,183608.314843,"POLYGON ((-8322903.570 1220895.451, -8322868.0..."
1,ATLÁNTICO,SOLEDAD,08,758,1,000,0001,02,07,0875810000000000010207,...,130.953499,9,8,0,La Esperanza,Oriental,{0AF04C46-B2B5-4076-B079-3E0D12A4BCBF},130.953763,990.019594,"POLYGON ((-8322730.482 1221783.629, -8322733.3..."
2,ATLÁNTICO,SOLEDAD,08,758,1,000,0001,02,08,0875810000000000010208,...,214.652178,69,68,0,La Esperanza,Oriental,{18C2D6D8-8470-4840-B4EC-458464DE9BEA},214.652283,2276.638460,"POLYGON ((-8322960.593 1221835.108, -8322960.9..."
3,ATLÁNTICO,SOLEDAD,08,758,1,000,0001,02,09,0875810000000000010209,...,200.189529,52,51,0,La Esperanza,Oriental,{71A178C7-9C94-48CF-97A0-57753AFB1132},200.189630,1987.208929,"POLYGON ((-8322924.501 1221838.043, -8322925.3..."
4,ATLÁNTICO,SOLEDAD,08,758,1,000,0001,02,10,0875810000000000010210,...,212.325244,69,68,0,La Esperanza,Oriental,{0825E96A-010B-438A-B97E-8C0B3D9701EB},212.325238,2156.185652,"POLYGON ((-8322893.023 1221826.126, -8322893.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5959,ATLÁNTICO,SOLEDAD,08,758,1,000,0041,08,22,0875810000000000410822,...,194.292154,37,36,0,Las Colonias,Centro Norte,{9B37C570-25E8-46F0-8BC5-D95B02D5AB3F},194.292128,1795.402218,"POLYGON ((-8325748.386 1222319.578, -8325759.5..."
5960,ATLÁNTICO,SOLEDAD,08,758,1,000,0041,08,23,0875810000000000410823,...,199.756804,25,24,0,Las Colonias,Centro Norte,{DE176271-02BD-4D3A-BA0C-D7668BBBE91E},199.756990,1241.110047,"POLYGON ((-8325610.961 1222366.767, -8325610.9..."
5961,ATLÁNTICO,SOLEDAD,08,758,1,000,0041,08,24,0875810000000000410824,...,226.386854,70,69,0,Las Colonias,Centro Norte,{08202CBF-BF84-4F60-B250-90F70B7A33C3},226.387268,1651.008027,"POLYGON ((-8325704.762 1222301.994, -8325701.8..."
5962,ATLÁNTICO,SOLEDAD,08,758,1,000,0041,08,25,0875810000000000410825,...,124.489530,15,14,0,Las Colonias,Centro Norte,{17CCE116-8E40-4345-9E57-CD7A49188CA2},124.489611,865.659502,"POLYGON ((-8325728.881 1222282.952, -8325726.4..."


In [19]:
people['SEXO_TOTAL'].sum()

534786

In [20]:
inhabitants = people['SEXO_TOTAL'].sum()

In [71]:
len(people['SEXO_TOTAL'])

5964

In [73]:
# utils functions

def calculate_contacts(polygon, points, date):
    _people = points[points.geometry.centroid.within(polygon.geometry.buffer(100))]
    data = {
        'date': np.repeat(date, len(_people)),
        'patient1': np.repeat(polygon['patient'], len(_people)),
        'patient2': list(_people['patient'])
    }
    
    exposure_df = pd.DataFrame(data=data, columns = ["date","patient1","patient2"])
    return exposure_df

# def extrapolation(df):
#     transformation_table = []
#     for i, value in df.sort_values("patient").iterrows():
#         original_index = value['patient']
#         new_index = 0
#         step = value['SEXO_TOTAL']
#         transformation_table[original_index] = [np.arange(new_index,new_index+step-1)]
#         new_index = new_index + step
#     return transformation_table

In [79]:
shapefile = "./shp/Barrios_info_DANE.shp"  
sim = dict()
sim["map"] = gdp.read_file(shapefile)
people = gdp.read_file("./shp/Censo_personas_manzanas_2018.shp", index=None) 
population = sim["map"]['SEXO_TOT_1']

N0 = len(people)
T = 1000
people['patient'] = np.arange(len(people))
#N0 = people.SEXO_TOTAL.sum()
#people = people.reindex(people.index.repeat(people.SEXO_TOTAL))

people_proj = people.to_crs({"init": "EPSG:3857"})

lat = people_proj.centroid.y
lng = people_proj.centroid.x

people_proj['calculated_centroid'] = people_proj.centroid

sim["location"] = pd.DataFrame(
{
    "patient": np.repeat(np.arange(N0), T),
    "date": np.tile(np.arange(T), N0),
    "latitude": np.repeat(lat, T),
    "longitude": np.repeat(lng, T),
    }
)

people_points = people_proj.centroid

distance_cutoff = 0.012
sim["patients"] = pd.DataFrame({"patient": np.unique(sim["location"]["patient"])})
sim["dates"] = pd.DataFrame({"date": np.unique(sim["location"]["date"])})

In [ ]:
# calculate daily contacts for day 1 as they will be repeated for the other days
date = sim["dates"]["date"][0]
daily_contacts = people_proj.apply(calculate_contacts, axis=1, args=[people_proj, date])
original_daily_contacts = pd.concat(list(daily_contacts), sort=False)
original_daily_contacts.reset_index(drop=True, inplace=True)

# calculate the table for all the dates
improved_daily_contacts = original_daily_contacts.copy()
for date in sim["dates"]["date"]:
    if len(original_daily_contacts[original_daily_contacts['date'] == date]) < 1:
        new_df = original_daily_contacts.copy()
        new_df['date'] = date
        improved_daily_contacts = improved_daily_contacts.append(new_df)

improved_daily_contacts.reset_index(drop=True, inplace=True)
improved_daily_contacts

sim["contacts"] = improved_daily_contacts
sim["N_c"] = contacts.calculate_Nc(sim)
print("Average daily contacts: {}".format(sim["N_c"]))

In [ ]:
# Run sim
sim["states"], sim["tests"] = state.simulate_states(sim, N_infected=15)
sim["hospital"] = state.get_first_occurrence(sim["states"], 6)
sim["deaths"] = state.get_first_occurrence(sim["states"], 8)

In [ ]:
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(
        df["longitude"],
        df["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

# 10 records
filtered_df

filtered_gdf = gpd.GeoDataFrame(
    filtered_df, 
    geometry=gpd.points_from_xy(
        filtered_df["longitude"],
        filtered_df["latitude"],
    ),
    crs={"init":"EPSG:4326"},
)

# EPSG:3857 converts it to meters, correct?

gdf_proj = gdf.to_crs({"init": "EPSG:3857"})
filtered_gdf_proj = filtered_gdf.to_crs({"init": "EPSG:3857"})

# so 100 miles would be 160934 meters

x = filtered_gdf_proj.buffer(100).unary_union

neighbours = gdf_proj["geometry"].intersection(x)